In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import ipywidgets as widgets

from IPython.display import display

In [6]:
# TODO: Should implement Enum for chart_period
# class ChartPeriod(Enum)
class TurtleProperties:
    """
    Holds the properties of the Turtle Trading Strategy

    Keyword Arguments:
    ticker -- Stock ticker
    chart_period -- Valid periods: "1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"
    moving_avg_period -- Number of days for moving average and average true range
    atr_multiple_stop_loss -- Multiple of ATR to use for stop loss
    atr_multiple_for_positions -- Multiple of ATR to use for adding to position
    risk -- Fraction of equity to risk per trade
    
    """
    def __init__(self, ticker, chart_period, moving_avg_period, atr_multiple_stop_loss, atr_multiple_for_positions, risk) -> None:
        self.ticker = ticker 
        self.chart_period = chart_period
        self.moving_avg_period = moving_avg_period
        self.atr_multiple_stop_loss = atr_multiple_stop_loss
        self.atr_multiple_for_positions = atr_multiple_for_positions
        self.risk = risk

In [7]:
output = widgets.Output()

In [ ]:
# Calculate the True Range for each day
true_range = pd.DataFrame({
    "TR1": abs(data["High"] - data["Low"]),
    "TR2": abs(data["High"] - data["Adj Close"].shift(1)),
    "TR3": abs(data["Low"] - data["Adj Close"].shift(1))
})
data["TR"] = true_range.max(axis=1)

# Calculate the Average True Range (ATR)
data["ATR"] = data["TR"].rolling(window=N+1).mean()
display(data)

In [ ]:
# Calculate the entry and exit points for long positions
data["long_entry"] = data["High"].rolling(window=N+1).max().shift(1)
data["long_exit"] = data["Low"].rolling(window=N+1).min().shift(1)

# Calculate the entry and exit points for short positions
data["short_entry"] = data["Low"].rolling(window=N+1).min().shift(1)
data["short_exit"] = data["High"].rolling(window=N+1).max().shift(1)
display(data)

In [ ]:
# Create a DataFrame to store the trading signals and the cumulative returns
signals = pd.DataFrame(index=data.index)
signals["signal"] = 0.0
signals["signal"] = np.where(data["Adj Close"] > data["long_entry"], 1.0, signals["signal"])
signals["signal"] = np.where(data["Adj Close"] < data["short_entry"], -1.0, signals["signal"])
signals["positions"] = signals["signal"].diff()
signals["returns"] = np.log(data["Adj Close"]/data["Adj Close"].shift(1))
display(signals)

In [ ]:
# Backtest the trading signals
initial_capital = float(100000.0)
positions = pd.DataFrame(index=signals.index).fillna(0.0)
positions["RIG"] = 100 * signals["signal"]
portfolio = positions.multiply(data["Adj Close"], axis=0)
pos_diff = positions.diff()
portfolio["holdings"] = (positions.multiply(data["Adj Close"], axis=0)).sum(axis=1)
portfolio["cash"] = initial_capital - (pos_diff.multiply(data["Adj Close"], axis=0)).sum(axis=1).cumsum()
portfolio["total"] = portfolio["cash"] + portfolio["holdings"]
portfolio["returns"] = portfolio["total"].pct_change()
# Print the final portfolio value and the Sharpe ratio
print("Final Portfolio Value: ${:.2f}".format(portfolio["total"].iloc[-1]))

In [ ]:
# indicates how much return an investor is receiving for each fractional_equity of risk taken
print("Sharpe Ratio: {:.2f}".format((portfolio["returns"].mean() / portfolio["returns"].std()) * np.sqrt(252)))
data['ATR'].ewm

In [16]:
# Load historical data
def run(btn, turtle: TurtleProperties):
    with output:
        output.clear_output()
        data = yf.download(turtle.ticker, period=turtle.chart_period)
        display(data)

In [19]:
# User Interface setup
ticker_widget = widgets.Text(placeholder = "Ticker")
chart_period_widget = widgets.Dropdown(
    options = ["1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"],
    placeholder = "Chart Period")
moving_avg_period_widget = widgets.Text(placeholder = "Period of Days for Moving Averages")
top_box = widgets.Box([ticker_widget, chart_period_widget, moving_avg_period_widget])

atr_multiple_stop_loss_widget = widgets.Text(placeholder = "Multiple of ATR for Stop Loss")
atr_multiple_for_positions_widget = widgets.Text(placeholder = "Multiple of ATR for Additional Positions")
risk_widget = widgets.Text(placeholder = "Risk per Trade")
bottom_box = widgets.Box([atr_multiple_stop_loss_widget, atr_multiple_for_positions_widget, risk_widget])

vert_box = widgets.VBox([top_box, bottom_box])
turtle = TurtleProperties(
    ticker_widget.value, 
    chart_period_widget.value, 
    moving_avg_period_widget.value, 
    atr_multiple_stop_loss_widget.value, 
    atr_multiple_for_positions_widget.value, 
    risk_widget.value)
button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
)

display(vert_box)
display(button)

button.on_click(run)
display(output)

Button(description='Submit', style=ButtonStyle(), tooltip='Submit')

Output(outputs=({'name': 'stdout', 'text': '[*********************100%***********************]  0 of 0 complet…

TypeError: run() missing 1 required positional argument: 'turtle'